In [12]:
#Mise en place de la dataframe Pandas (exécution environ 10s)

import math
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
import io

json_url = "https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/exports/json?lang=fr&timezone=Europe%2FParis&select=equip_nom%2C%20equip_service_date%2C%20coordonnees%2C%20equip_x%2C%20equip_y&where=reg_nom%20like%20%22Ile%20de%20France%22"
req = requests.get(json_url)
json_data = req.json()
df_dirty = pd.DataFrame(json_data)
df_dirty['equip_service_date'] = pd.to_numeric(df_dirty['equip_service_date'], errors='coerce')
df = df_dirty.dropna(subset=['equip_service_date'])

# Les coordonnées sont au format WGS84 d'après la documentation. On définit donc le CRS en adéquation.
geodf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['equip_x'], df['equip_y']), crs="EPSG:4326")  

In [13]:
#Test
geodf.head(10)

,equip_nom,equip_service_date,coordonnees,equip_x,equip_y,geometry
1,TERRAIN DE FOOTBALL SYNTHETIQUE,2009.0,"{'lon': 2.61006, 'lat': 48.98095}",2.610060,48.980950,POINT (2.61006 48.98095)
2,PISCINE,1975.0,"{'lon': 2.61158, 'lat': 48.98034}",2.611580,48.980340,POINT (2.61158 48.98034)
3,GRANDE SALLE,2000.0,"{'lon': 2.59892, 'lat': 48.96226}",2.598920,48.962260,POINT (2.59892 48.96226)
6,PLATEAU EPS,1994.0,"{'lon': 2.624200880527497, 'lat': 48.976618801...",2.624201,48.976619,POINT (2.6242 48.97662)
11,Salle polyvalente,1960.0,"{'lon': 2.6031777262687688, 'lat': 48.95546702...",2.603178,48.955467,POINT (2.60318 48.95547)
13,COURT DE TENNIS COUVERT 1,1982.0,"{'lon': 2.5996854901313786, 'lat': 48.62961867...",2.599685,48.629619,POINT (2.59969 48.62962)
14,TERRAIN DE FOOTBALL D'HONNEUR ENTRAINEMENT,1982.0,"{'lon': 2.59818, 'lat': 48.62985}",2.598180,48.629850,POINT (2.59818 48.62985)
15,TERRAIN DE FOOTBAL N°3,1993.0,"{'lon': 2.59821, 'lat': 48.63171}",2.598210,48.631710,POINT (2.59821 48.63171)
16,TERRAIN DE FOOTBALL (en stabilisé),1982.0,"{'lon': 2.59407, 'lat': 48.62991}",2.594070,48.629910,POINT (2.59407 48.62991)
17,COURT DE TENNIS COUVERT 2,1982.0,"{'lon': 2.599830329418183, 'lat': 48.629359851...",2.599830,48.629360,POINT (2.59983 48.62936)


In [30]:
#Recherche de l'infrastructure la plus proche de (x,y) en exploitant les opérations vectorisées de Pandas :
def closest_infra(x,y) :
    """
    Trouve l'infrastructure la plus proche de (x, y) dans un GeoDataFrame WGS84.

    Arguments:
        x (float) : Longitude 
        y (float) : Latitude 

    Renvoie:
        L'entrée correspondant à l'infrastructure la plus proche
    """
    point = Point(x,y) # Coord. en format WGS84
    geodf_proj = geodf.to_crs(epsg=3857) 
    point_proj = gpd.GeoSeries([point], crs="EPSG:4326").to_crs(epsg=3857)[0] # Reprojeter les coordonnées (format WGS84) en système métrique pour augmenter la précision
    geodf_proj['distance'] = geodf_proj.geometry.distance(point_proj) # Calcul des distances entre notre point et toutes les infrastructures, grâce à GeoPandas
    indice_closest = geodf_proj['distance'].idxmin() # Retourner l'indice du minimul avec idxmin
    return geodf.loc[indice_closest]

In [31]:
closest_infra(2.422434324, 48.54397854)

equip_nom                           PLATEAU D'EVOLUTION
equip_service_date                               1975.0
coordonnees           {'lon': 2.42674, 'lat': 48.54706}
equip_x                                         2.42674
equip_y                                        48.54706
geometry                       POINT (2.42674 48.54706)
Name: 18387, dtype: object

In [32]:
closest_infra(1.422434324, 45.54397854)

equip_nom                           TERRAIN DE FOOTBALL
equip_service_date                               1999.0
coordonnees           {'lon': 2.61341, 'lat': 48.14217}
equip_x                                         2.61341
equip_y                                        48.14217
geometry                       POINT (2.61341 48.14217)
Name: 34, dtype: object